In [0]:
%run /Workspace/Users/loganathansundaramsa@gmail.com/dataengg_inc_learning/databricks_learning/Brazilian_Ecommerce/gendral_config/com_utl_fun

In [0]:
import json

gendral_config = dbutils.notebook.run("/Workspace/Users/loganathansundaramsa@gmail.com/dataengg_inc_learning/databricks_learning/Brazilian_Ecommerce/gendral_config/common_notebook", 120)


config_dict = json.loads(gendral_config)

src = config_dict['src']
bronze = config_dict['bronze']
silver = config_dict['silver']
gold = config_dict['gold']
catalog = config_dict['CATALOG']
schema = config_dict['SCHEMA']

In [0]:
df_products = read_file(spark,"parquet",f'{bronze}/products')
df_dedup = deduplicate(df_products,["product_id"])
df_stand = standardize_string_columns(df_dedup,{"product_category_name":"lower_trim"})
# df_stand.display()
columns = {
        "product_weight_g": "integer",
        "product_length_cm":"integer",
        "product_height_cm":"integer",
        "product_width_cm":"integer"}

df_cast = f_cast_col(df_stand, columns)

df_filter_1 = df_cast.filter(""" product_weight_g > 0 
                              AND product_length_cm > 0 
                              AND product_height_cm > 0
                              AND product_width_cm > 0""") \
                            .withColumn("product_volume",col("product_length_cm")*col("product_height_cm")*col("product_width_cm"))

columns = {
        "created_at": {"type": "timestamp"},
        "updated_at": {"type": "timestamp"},
        "source": {"type": "string", "default": "products"}}

df_final_products = enrich_add_col(df_filter_1, columns)

# df_final_products.display()

In [0]:

# Write data delta file /delta table in silver location

write_file(df_final_products,f"{silver}/silver_products")

write_delta_liquid(df_final_products, f"{catalog}.{schema}.silver_products",["product_id","product_category_name"])